In [ ]:
using ForwardDiff

In [ ]:
using JLD2
using FileIO
using PyPlot
using Statistics
using StatsBase 
using Distributions
using LinearAlgebra
using HDF5
using BenchmarkTools

In [ ]:
using IntervalSets
using Random, ArraysOfArrays
using ValueShapes
using Measurements

In [ ]:
# using Revise
using BAT 

In [ ]:
using Plots

In [ ]:
conv_mat = load("../data/experiment/dataset_2/m2/conv-matrix-upd-1.jld2")

conv_matrices = (
    cam_1 = conv_mat["cam_1"],
    cam_2 = conv_mat["cam_2"],
    cam_3 = conv_mat["cam_3"],
    cam_4 = conv_mat["cam_4"], 
) 

images = load("../data/experiment/dataset_2/m2/images-satur.jld2");
event_number = 440

# Large Charge: Ev_2_2 = 2, Ev_2_1 = 311, Ev_2_3 = 440

event = (
    cam_1 = images["cam_1"][event_number,:,:],
    cam_2 = images["cam_2"][event_number,:,:],
    cam_3 = images["cam_3"][event_number,:,:],
    cam_4 = images["cam_4"][event_number,:,:],
    population = images["charge"][event_number,:][1],
);


In [ ]:
function likelihood_cam4(
        params::T, 
        image::Array{F,2},
        population::AbstractFloat,
        cam_ind::Integer;
        n_threads::Integer = Threads.nthreads()
    ) where {T <: NamedTuple, F <: AbstractFloat}
   
    VT = eltype(params.tr_size)
    
    tot_loglik = zeros(VT, n_threads)  
    
    light_coefficient = params.cam4_light_amp * 10^5
    
    δ_x = params.cam4_psx * 10^-3
    δ_y = params.cam4_psy * 10^-3
    
    μ_x  = params.algmx[cam_ind] * δ_x
    μ_y  = params.algmy[cam_ind] * δ_y
    
    σ_x = sqrt.(params.tr_size[1]^2 + 10^-4*params.ang_spr[1]^2*(params.waist[1] - params.s_cam[cam_ind])^2) 
    σ_y = sqrt.(params.tr_size[2]^2 + 10^-4*params.ang_spr[2]^2*(params.waist[1] - params.s_cam[cam_ind])^2) 

    σ_x = sqrt(σ_x^2 + (params.cam4_resx*δ_x).^2)
    σ_y = sqrt(σ_y^2 + (params.cam4_resy*δ_y).^2)
    
    function pixel_log_lik(cart_ind)
        
        log_lik = zero(VT)
        
        if !isnan(image[cart_ind])
            
            x_edge = cart_ind.I[1] * δ_x
            y_edge = cart_ind.I[2] * δ_y

            pix_prediction = cdf(Normal(μ_x,σ_x), x_edge) - cdf(Normal(μ_x,σ_x), x_edge - δ_x)
            pix_prediction *= cdf(Normal(μ_y,σ_y), y_edge) - cdf(Normal(μ_y,σ_y), y_edge - δ_y)
            pix_prediction = pix_prediction*light_coefficient + params.cam4_ped

            if pix_prediction > 10^4
                pix_prediction = 10^4 
            end   

            log_lik = logpdf(truncated(Normal(pix_prediction, params.cam4_light_fluct*sqrt(pix_prediction)), 0.0, 4096), image[cart_ind])
        end

        return log_lik
    end
    
    Threads.@threads for t = 1:n_threads
        tot_loglik[t] = sum(broadcast(x -> pixel_log_lik(x), CartesianIndices(image)[t:n_threads:length(image)] ))
    end

    return sum(tot_loglik)
end

function background_conv(cv_matrix::Array{Float64,2}, observed::Int64, expected::T) where {T <: Real}
    
    VT = typeof(expected)
    
    expected = expected + 1 # convert into matrix index 
    observed = observed + 1

    left_exp, right_exp = floor(Int64, expected), ceil(Int64, expected)
    
    if left_exp != right_exp
        left_prob, right_prob = exp(cv_matrix[observed, left_exp]), exp(cv_matrix[observed, right_exp])
        int_prob = log(left_prob + (right_prob - left_prob)*(expected - left_exp))
    else 
       int_prob =  cv_matrix[observed, left_exp]
    end
    
    return int_prob   
end

function likelihood_cam13(
        params::T, 
        image::Array{Float64,2},
        population::Float64,
        cv_matrix::Array{Float64,2},
        cam_ind::Int64;
        n_threads = Threads.nthreads()
    ) where {T <: NamedTuple}
    
    
    VT = typeof(params.tr_size[1])
    
    tot_loglik = zeros(VT, n_threads)
    light_coefficient::VT = params.light_amp[cam_ind] * 10^5
    
    δ_x::VT = params.psx[cam_ind] * 10^-3
    δ_y::VT = params.psy[cam_ind] * 10^-3
    
    μ_x::VT  = params.algmx[cam_ind] * δ_x
    μ_y::VT  = params.algmy[cam_ind] * δ_y
    
    σ_x::VT = sqrt.(params.tr_size[1]^2 + 10^-4*params.ang_spr[1]^2*(params.waist[1] - params.s_cam[cam_ind])^2) 
    σ_y::VT = sqrt.(params.tr_size[2]^2 + 10^-4*params.ang_spr[2]^2*(params.waist[1] - params.s_cam[cam_ind])^2) 
    
    σ_x = sqrt(σ_x^2 + (params.resx[cam_ind]*δ_x).^2)
    σ_y = sqrt(σ_y^2 + (params.resy[cam_ind]*δ_y).^2) # \sigma x is the same for both
    
    max_pred_amp::Int64 = size(cv_matrix)[2]-1
    
    Threads.@threads for t = 1:n_threads
        
        cum_log_lik = zero(VT)
        
        for pix_ind in CartesianIndices(image)[t:n_threads:length(image)] 
            if !isnan(image[pix_ind])
                x_edge::VT = pix_ind.I[1] * δ_x
                y_edge::VT = pix_ind.I[2] * δ_y

                pix_prediction = cdf(Normal(μ_x,σ_x), x_edge) - cdf(Normal(μ_x,σ_x), x_edge - δ_x)
                pix_prediction *= cdf(Normal(μ_y,σ_y), y_edge) - cdf(Normal(μ_y,σ_y), y_edge - δ_y)

                cv_index = pix_prediction*light_coefficient
                
                if cv_index > max_pred_amp - 1
                    cv_index = max_pred_amp - 1
                end
                
                cum_log_lik += background_conv(cv_matrix, Int64(image[pix_ind]), cv_index) # interpolated convolution 
            end
        end
        
        tot_loglik[t] = cum_log_lik
        
    end

    return sum(tot_loglik)
end

In [ ]:
param_truth = (
    tr_size = [0.1407294407925774, 0.137012154400229], 
    ang_spr = [3.960531197813597, 4.1815937347856025], 
    waist = [2.921223603216503], 
    algmx = [33.72219850943121, 34.546617378830824, 20.634013004011102, 35.43220219068294], 
    algmy = [36.063422948400145, 37.37048076467348, 21.835681202303704, 34.94100216204202], 
    cam4_ped = 32.754991551558724, 
    cam4_light_fluct = 2.1913563229061204, 
    cam4_light_amp = 8.824954442678175, 
    resx = [0.8051569342140491, 0.540541189498069, 0.7342548245379088], 
    resy = [3.7794370837087508, 0.6186676377752929, 2.565553468252872], 
    cam4_resx = 1.339082789468911, 
    cam4_resy = 1.2429021486346863, 
    psx = [27.123268870682487, 21.89402537577619, 115.56357977889763], 
    psy = [29.323407884013157, 23.814076209426148, 124.38805051756697], 
    cam4_psx = 90.65358820186805, 
    cam4_psy = 90.67947529363211, 
    light_amp = [8.738368829879834, 11.950987284247667, 2.7858070849651555], 
    s_cam = [0.0, 1.478, 15.026, 23.115]
)

# Gradients Calculation: 

In [ ]:
# loglik = likelihood_cam13(param_truth, event.cam_1, event.population, conv_matrices.cam_1, 1)
# loglik = likelihood_cam13(param_truth, event.cam_2, event.population, conv_matrices.cam_2, 2)
# loglik = likelihood_cam13(param_truth, event.cam_3, event.population, conv_matrices.cam_3, 3)
loglik = likelihood_cam4(param_truth, event.cam_4, event.population, 4)

In [ ]:
@benchmark likelihood_cam4($param_truth, $event.cam_4, $event.population, 4)

In [ ]:
# using Revise, Profile, PProf
# @profile likelihood_cam4(param_truth, event.cam_4, event.population, 4)
# pprof(;webport=58599)

In [ ]:
function logd(x; c1=true, c2=true, c3=true, c4=true)
    
    param_run = (
        tr_size = [x[1], x[2]],
        ang_spr = [x[3], x[4]],
        waist = [x[5],],
        algmx = [x[6], x[7], x[8], x[9]], 
        algmy = [x[10], x[11], x[12], x[13]], 
        cam4_ped = x[14], 
        cam4_light_fluct = x[15], 
        cam4_light_amp = x[16], 
        resx = [x[17], x[18], x[19]], 
        resy = [x[20], x[21], x[22]], 
        cam4_resx = x[23], 
        cam4_resy = x[24], 
        psx = [x[25], x[26], x[27]], 
        psy = [x[28], x[29], x[30]], 
        cam4_psx = x[31], 
        cam4_psy = x[32], 
        light_amp = [x[33], x[34], x[35]], 
        s_cam = [0.0, 1.478, 15.026, 23.115]
    )
    
    ll = 0.0
    
    if c1 
        ll += likelihood_cam13(param_run, event.cam_1, event.population, conv_matrices.cam_1, 1)
    end
    
    if c2 
        ll += likelihood_cam13(param_run, event.cam_2, event.population, conv_matrices.cam_2, 2)
    end
    
    if c3
        ll += likelihood_cam13(param_run, event.cam_3, event.population, conv_matrices.cam_3, 3)
    end
    
    if c4 
        ll += likelihood_cam4(param_run, event.cam_4, event.population, 4)
    end

    
    return ll

end

In [ ]:
ld1(x) = logd(x; c1=true, c2=false, c3=false, c4=false)
grad1(x) = ForwardDiff.gradient(ld1, x)
ld2(x) = logd(x; c1=false, c2=true, c3=false, c4=false)
grad2(x) = ForwardDiff.gradient(ld2, x)
ld3(x) = logd(x; c1=false, c2=false, c3=true, c4=false)
grad3(x) = ForwardDiff.gradient(ld3, x)
ld4(x) = logd(x; c1=false, c2=false, c3=false, c4=true)
grad4(x) = ForwardDiff.gradient(ld4, x)
ld_all(x) = logd(x; c1=true, c2=true, c3=true, c4=true)
grad_all(x) = ForwardDiff.gradient(ld_all, x)

In [ ]:
x_0 = [
    0.1407294407925774, 
    0.137012154400229, 
    3.960531197813597, 
    4.1815937347856025, 
    2.921223603216503, 
    33.72219850943121, 
    34.546617378830824, 
    20.634013004011102, 
    35.43220219068294, 
    36.063422948400145, 
    37.37048076467348, 
    21.835681202303704, 
    34.94100216204202, 
    32.754991551558724, 
    2.1913563229061204, 
    8.824954442678175, 
    0.8051569342140491, 
    0.540541189498069, 
    0.7342548245379088, 
    3.7794370837087508, 
    0.6186676377752929, 
    2.565553468252872, 
    1.339082789468911, 
    1.2429021486346863, 
    27.123268870682487, 
    21.89402537577619, 
    115.56357977889763, 
    29.323407884013157, 
    23.814076209426148, 
    124.38805051756697, 
    90.65358820186805, 
    90.67947529363211, 
    8.738368829879834, 
    11.950987284247667, 
    2.7858070849651555
];

In [ ]:
grad_all(x_0)

In [ ]:
@benchmark ld_all(x_0)

In [ ]:
@benchmark grad_all(x_0)

In [ ]:
# f_t(x) = pdf(Normal(1, 2), x[1])*pdf(Normal(1, 1), x[2])
# gg(x) = ForwardDiff.gradient(f_t, x)

In [ ]:
xrange=range(0.0, stop = 0.3, length=25)
yrange=range(0.0, stop = 0.35, length=25)

x_coor = [x for x in xrange, y in yrange]'
y_coor = [y for x in xrange, y in yrange]';


In [ ]:
cam1_z =  [ld1([x,y, x_0[3:end]...]) for x in xrange, y in yrange]'
cam1_zx = [grad1([x, y, x_0[3:end]...])[1] for x in xrange, y in yrange]'
cam1_zy = [grad1([x, y, x_0[3:end]...])[2] for x in xrange, y in yrange]'
cam1_speed = sqrt.(cam1_zx.^2 .+ cam1_zy.^2);

In [ ]:
cam2_z =  [ld2([x,y, x_0[3:end]...]) for x in xrange, y in yrange]'
cam2_zx = [grad2([x,y, x_0[3:end]...])[1] for x in xrange, y in yrange]'
cam2_zy = [grad2([x,y, x_0[3:end]...])[2] for x in xrange, y in yrange]'
cam2_speed = sqrt.(cam2_zx.^2 .+ cam2_zy.^2);

In [ ]:
cam3_z =  [ld3([x,y, x_0[3:end]...]) for x in xrange, y in yrange]'
cam3_zx = [grad3([x,y, x_0[3:end]...])[1] for x in xrange, y in yrange]'
cam3_zy = [grad3([x,y, x_0[3:end]...])[2] for x in xrange, y in yrange]'
cam3_speed = sqrt.(cam3_zx.^2 .+ cam3_zy.^2);

In [ ]:
cam4_z =  [ld4([x,y, x_0[3:end]...]) for x in xrange, y in yrange]'
cam4_zx = [grad4([x,y, x_0[3:end]...])[1] for x in xrange, y in yrange]'
cam4_zy = [grad4([x,y, x_0[3:end]...])[2] for x in xrange, y in yrange]'
cam4_speed = sqrt.(cam4_zx.^2 .+ cam4_zy.^2);

In [ ]:
cam5_z =  [ld_all([x,y, x_0[3:end]...]) for x in xrange, y in yrange]'
cam5_zx = [grad_all([x,y, x_0[3:end]...])[1] for x in xrange, y in yrange]'
cam5_zy = [grad_all([x,y, x_0[3:end]...])[2] for x in xrange, y in yrange]'
cam5_speed = sqrt.(cam5_zx.^2 .+ cam5_zy.^2);

In [ ]:
fig, ax = subplots(1,1, figsize=(6,6))

lw = 4 .* cam5_speed ./ maximum(cam5_speed) # Line Widths
ax.contourf(xrange, yrange, cam5_z, cmap="Spectral_r", levels=20)
ax.streamplot(x_coor,y_coor,cam5_zx,cam5_zy, density=0.8,color="k",linewidth=lw)

ax.set_xlim(minimum(xrange), maximum(xrange))
ax.set_ylim(minimum(yrange), maximum(yrange))

ax.set_title("Camera (#1-4)")

ax.set_xlabel(L"\sigma_x")
ax.set_ylabel(L"\sigma_y")

In [ ]:
# fig, ax = subplots(1,1, figsize=(6,6))

# lw = 6 .* cam1_speed ./ maximum(cam1_speed) # Line Widths
# ax.contourf(xrange, yrange, cam1_z, cmap="Spectral_r", levels=20)
# ax.streamplot(x_coor,y_coor,cam1_zx,cam1_zy, density=0.8,color="k",linewidth=lw)

# ax.set_xlim(minimum(xrange), maximum(xrange))
# ax.set_ylim(minimum(yrange), maximum(yrange))

# ax.set_title("Camera #1")

In [ ]:
fig, ax = subplots(1,1, figsize=(8,8))

lw = 2 .* cam5_speed ./ maximum(cam5_speed) # Line Widths
ax.contourf(xrange, yrange, cam5_z, cmap="Spectral_r", levels=20, alpha=0.8)
ax.contour(xrange, yrange, cam1_z, cmap="Blues", levels=10, alpha=0.5, linestyles="--")
ax.contour(xrange, yrange, cam2_z, cmap="Oranges", levels=10, alpha=0.5, linestyles="--")
ax.contour(xrange, yrange, cam3_z, cmap="Greens", levels=10, alpha=0.5, linestyles="--")
ax.contour(xrange, yrange, cam4_z, cmap="Reds", levels=10, alpha=0.5, linestyles="--")
# ax.streamplot(x_coor,y_coor,cam5_zx,cam5_zy, density=0.7,color="k",linewidth=lw)

ax.set_xlim(minimum(xrange), maximum(xrange))
ax.set_ylim(minimum(yrange), maximum(yrange))

ax.set_title("Camera All")

# MCMC:

In [ ]:
β = 0.015

# prior = NamedTupleDist(
#         tr_size = [truncated(Normal(0.2, 0.04), 0.08, 0.25), truncated(Normal(0.2, 0.04), 0.08, 0.25)],
#         ang_spr = [truncated(Normal(4.0, 2.0), 2.6, 5.0), truncated(Normal(4.0, 2.0), 2.6, 5.0)], # changed prior range
#         waist = [Normal(2.9, 0.03)],
#         algmx = [23.0 .. 48, 23.0 .. 48.0, 10.0 .. 30.0, 23.0 .. 48.0],
#         algmy = [23.0 .. 48, 23.0 .. 48.0, 10.0 .. 30.0, 23.0 .. 48.0],
#         cam4_ped = 4.0 .. 40.0,
#         cam4_light_fluct = 1.0 .. 3.0,
#         cam4_light_amp = 1.6 .. 9.9, 
#         resx = [truncated(Normal(1, 0.5), 0, Inf), truncated(Normal(1, 0.5), 0, Inf), truncated(Normal(1, 0.5), 0, Inf)], 
#         resy = [truncated(Normal(1, 0.5), 0, Inf), truncated(Normal(1, 0.5), 0, Inf), truncated(Normal(1, 0.5), 0, Inf)], 
#         cam4_resx = truncated(Normal(1, 0.5), 0, Inf),
#         cam4_resy = truncated(Normal(1, 0.5), 0, Inf),
#         psx = [truncated(Normal(27.1, 27.1*β), 0., Inf), truncated(Normal(21.6, 21.6*β), 0., Inf), truncated(Normal(114.0, 114.0*β), 0., Inf)], # pixels are in microns
#         psy = [truncated(Normal(30.5, 30.5*β), 0., Inf), truncated(Normal(23.4, 23.4*β), 0., Inf), truncated(Normal(125.0, 125.0*β), 0., Inf)],
#         cam4_psx = truncated(Normal(91.0, 91.0*β), 0., Inf),
#         cam4_psy = truncated(Normal(89.4, 89.4*β), 0., Inf),
#         light_amp  = [1.0 .. 13.0 , 1.0 .. 17.0, 1.0 .. 5.0], 
#         s_cam = [0.0, 1.478, 15.026, 23.1150],

#     ); 

prior = NamedTupleDist(
        tr_size = [truncated(Normal(0.2, 0.04), 0.08, 0.25), truncated(Normal(0.2, 0.04), 0.08, 0.25)],
        ang_spr = [truncated(Normal(4.0, 2.0), 2.6, 5.0), truncated(Normal(4.0, 2.0), 2.6, 5.0)], # changed prior range
        waist = [Normal(2.9, 0.03)],
        algmx = [33.72219850943121, 34.546617378830824, 20.634013004011102, 35.43220219068294],
        algmy = [36.063422948400145, 37.37048076467348, 21.835681202303704, 34.94100216204202],
        cam4_ped = 32.754991551558724, 
        cam4_light_fluct = 2.1913563229061204, 
        cam4_light_amp = 8.824954442678175, 
        resx = [0.8051569342140491, 0.540541189498069, 0.7342548245379088], 
        resy = [3.7794370837087508, 0.6186676377752929, 2.565553468252872], 
        cam4_resx = 1.339,
        cam4_resy = 1.2429,
        psx = [27.1, 21.6, 114.0],
        psy = [30.5, 23.4, 125.0],
        cam4_psx = 91.0,
        cam4_psy = 89.4,
        light_amp  = [8.738368829879834, 11.950987284247667, 2.7858070849651555], 
        s_cam = [0.0, 1.478, 15.026, 23.1150],

    ); 

log_likelihood = let e = event, c = conv_matrices
    
    params -> begin
        
        ll = 0.0
        ll += likelihood_cam13(params, e.cam_1, e.population, c.cam_1, 1)
        ll += likelihood_cam13(params, e.cam_2, e.population, c.cam_2, 2)
        ll += likelihood_cam13(params, e.cam_3, e.population, c.cam_3, 3)
        ll += likelihood_cam4(params, e.cam_4, e.population, 4)
        
        return LogDVal(ll)
        
    end
end

posterior = PosteriorDensity(log_likelihood, prior);

## Likelihood Optimization: 

In [ ]:
log_likelihood(param_truth)

@benchmark log_likelihood(param_truth)

## BFGS:

In [ ]:
param_truth

In [ ]:
# findmode_result = bat_findmode(posterior, MaxDensityLBFGS(init = ExplicitInit([param_truth,])), )

# fit_par_values = findmode_result.result

In [ ]:
findmode_result.info

### MCMC Sampling:

In [ ]:
posterior = PosteriorDensity(log_likelihood, prior)

tuning = AdaptiveMHTuning(
    λ = 0.5,
    α = ClosedInterval(0.15,0.25),
    β = 1.5,
    c = ClosedInterval(1e-4,1e2),
    r = 0.5,
)

convergence = BrooksGelmanConvergence(
    threshold = 1.1,
    corrected = false
)

init = MCMCChainPoolInit(
    init_tries_per_chain = ClosedInterval(50,150),
    max_nsamples_init = 500,
    max_nsteps_init = 500,
    max_time_init = Inf
)

burnin = MCMCMultiCycleBurnin(
    max_nsamples_per_cycle = 4000,
    max_nsteps_per_cycle = 4000,
    max_time_per_cycle = Inf,
    max_ncycles = 120
)

nsamples = 2*10^3
nchains = 4

sampler = MetropolisHastings(tuning=tuning,)

algorithm = MCMCSampling(sampler=sampler, 
    nchains=nchains, 
    init=init, 
    burnin=burnin, 
    convergence=convergence
);

In [ ]:
samples = bat_sample(posterior, nsamples, algorithm, max_neval = nsamples, max_time = Inf).result

## HMC Sampling: 

In [ ]:
posterior = PosteriorDensity(log_likelihood, prior);

In [ ]:
posterior_is = bat_transform(PriorToGaussian(), posterior, PriorSubstitution()).result;

In [ ]:
iters = 1000
iters_warmup = 100
chains = 4

metric = BAT.DiagEuclideanMetric()
integrator = BAT.LeapfrogIntegrator(0.0)
proposal = BAT.NUTS(:MultinomialTS, :ClassicNoUTurn)
adaptor = BAT.StanHMCAdaptor(0.8, iters_warmup)

ahmc_sampler = HamiltonianMC(metric, ForwardDiff, integrator, proposal, adaptor);

In [ ]:
@time samples_is = bat_sample(posterior_is, iters, MCMCSampling(sampler = ahmc_sampler, nchains = chains)).result;

In [ ]:
# 2735.565987 / 1000 samples - 39.70% gc time) - 3 sec per sample

In [ ]:
bat_eff_sample_size(samples_is)

In [ ]:
trafo_is = trafoof(posterior_is.likelihood)
samples = inv(trafo_is).(samples_is);

In [ ]:
smpls_flat = flatview(unshaped.(samples.v))

plt.plot(smpls_flat[3,:])

In [ ]:
Plots.plot(samples)

In [ ]:
BAT.LeapfrogIntegrator